In [2]:
import pandas as pd

from typing import Optional, Union
import pandas as pd, numpy as np, torch
from dataclasses import dataclass
from transformers import AutoTokenizer, AutoConfig
from transformers import EarlyStoppingCallback
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer, RobertaForMultipleChoice
import argparse
from transformers import get_polynomial_decay_schedule_with_warmup, TrainerCallback
import datasets
from datasets import Dataset

In [3]:
class args:
    train_data = './dataset/demo_train.csv'
    MAX_INPUT = 1024

In [4]:
df_train = pd.read_csv(args.train_data).reset_index(drop = True)
#df_valid = pd.read_csv(args.valid_data).reset_index(drop = True)

In [21]:
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}

In [22]:
def get_label(row):
    label = [idx for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie']) if row[option] == 1]
    return label[-1]

In [23]:
df_train['label'] = df_train.apply(lambda x: get_label(x), axis = 1)

In [35]:
def preprocess(example):
    first_sentence = [ "[CLS] " + example['prompt'] ] * 2
    second_sentences = [" #### " + example['prompt'] + " [SEP] " + example[option] + " [SEP]" for option in ['response_a','response_b']]
    tokenized_example = tokenizer(first_sentence, second_sentences, truncation='longest_first', 
                                  max_length=args.MAX_INPUT, add_special_tokens=False)
    tokenized_example['label'] = example['label']
    return tokenized_example

In [5]:
def preprocess(example):
    sentences = [" #### " + example['prompt'] + " [SEP] " + example['response_a'] + " [SEP]" +  " #### " + example['prompt'] + " [SEP] " + example['response_b'] + " [SEP]"]
    tokenized_example = tokenizer(sentences, truncation=True, 
                                  max_length=args.MAX_INPUT, add_special_tokens=False)
    tokenized_example['label'] = example['label']
    return tokenized_example

In [6]:
dataset = datasets.Dataset.from_pandas(df_train)
MODEL = 'microsoft/deberta-v3-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/root/miniconda3/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [37]:
dataset

Dataset({
    features: ['Unnamed: 0', 'id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b', 'winner_model_a', 'winner_model_b', 'winner_tie', 'label'],
    num_rows: 1001
})

In [7]:
tokenized_dataset = dataset.map(preprocess, remove_columns=['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b'])# 

Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

In [13]:
tokenized_dataset

Dataset({
    features: ['Unnamed: 0', 'winner_model_a', 'winner_model_b', 'winner_tie', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1001
})

In [ ]:
tokenized_dataset['input_ids'][0]

In [32]:
df_train.loc[40]

Unnamed: 0                                                       40
id                                                          3258431
model_a                                     stablelm-tuned-alpha-7b
model_b                                                  vicuna-13b
prompt            ["The following is a command that the user is ...
response_a                                                  ["Yes"]
response_b                                                   ["NO"]
winner_model_a                                                    1
winner_model_b                                                    0
winner_tie                                                        0
label                                                             0
Name: 40, dtype: object

In [14]:
df_train.loc[:1000,].reset_index(drop = True).to_csv('demo_train.csv')

In [15]:
df_train.loc[1000:1200,].reset_index(drop = True).to_csv('demo_valid.csv')